In [6]:
import sys
sys.path.append('On_Multiple_Semantics_for_Declarative_Database_Repairs/')
import On_Multiple_Semantics_for_Declarative_Database_Repairs
print(sys.path)

['/Users/kushagraghosh/Desktop/DataRepairAndFairness', '/Users/kushagraghosh/opt/anaconda3/envs/hc36/lib/python36.zip', '/Users/kushagraghosh/opt/anaconda3/envs/hc36/lib/python3.6', '/Users/kushagraghosh/opt/anaconda3/envs/hc36/lib/python3.6/lib-dynload', '', '/Users/kushagraghosh/opt/anaconda3/envs/hc36/lib/python3.6/site-packages', '/Users/kushagraghosh/opt/anaconda3/envs/hc36/lib/python3.6/site-packages/IPython/extensions', '/Users/kushagraghosh/.ipython', 'On_Multiple_Semantics_for_Declarative_Database_Repairs/', '../', 'On_Multiple_Semantics_for_Declarative_Database_Repairs/', 'On_Multiple_Semantics_for_Declarative_Database_Repairs/']


In [7]:
from database_generator.dba import DatabaseEngine
from Semantics.end_sem import EndSemantics
from Semantics.stage_sem import StageSemantics
from Semantics.step_sem import StepSemantics
from Semantics.independent_sem import IndependentSemantics

In [9]:
# specify the schema
mas_schema = {"synthetic_data_version_1": ('RAC1P', 'SEX', 'REGION', 'PINCP', 'ST', 'COW', 'CIT', 'NATIVITY', 'MSP', 'SCHL', 'DIS','MIL')
              }


def read_rules(rule_file):
    """read programs from txt file"""
    all_programs = []
    with open(rule_file) as f:
        rules = []
        for line in f:
            if line.strip():
                tbl, r = line.split("|")
                rules.append((tbl, r[:-2]))
            else:
                all_programs.append([r for r in rules])
                rules = []
        all_programs.append(rules)
    return all_programs

# load delta programs
programs = read_rules("On_Multiple_Semantics_for_Declarative_Database_Repairs/../delta_program.txt")

# start the database
db = DatabaseEngine("cr")

tbl_names = ["synthetic_data_version_1"]

def database_reset(db):
    """reset the database"""
    res = db.execute_query("select current_database();")
    db_name = res[0][0]
    db.delete_tables(tbl_names)
    # db.close_connection()
    db = DatabaseEngine(db_name)
    db.load_database_tables(tbl_names)

# choose a delta program from the programs file
rules = programs[1]
print("delta program:", rules)

# # reset the database between runs
# database_reset(db)

# end_sem = EndSemantics(db, rules, tbl_names)
# end_semantics_result = end_sem.find_mss()
# print("result for end semantics:", end_semantics_result)

# # reset the database between runs
# database_reset(db)

# stage_sem = StageSemantics(db, rules, tbl_names)
# stage_semantics_result = stage_sem.find_mss()
# print("result for stage semantics:", stage_semantics_result)

# reset the database between runs
database_reset(db)

step_sem = StepSemantics(db, rules, tbl_names)
step_semantics_result = step_sem.find_mss(mas_schema)
print("result for step semantics:", step_semantics_result)

# reset the database between runs
# database_reset(db)

# ind_sem = IndependentSemantics(db, rules, tbl_names)
# ind_semantics_result = ind_sem.find_mss(mas_schema)
# print("result for independent semantics:", ind_semantics_result)


delta program: [('synthetic_data_version_1', 'SELECT synthetic_data_version_11.* FROM synthetic_data_version_1 AS synthetic_data_version_11, synthetic_data_version_1 AS synthetic_data_version_12 WHERE synthetic_data_version_11.CIT = synthetic_data_version_12.CIT AND synthetic_data_version_11.NATIVITY <> synthetic_data_version_12.NATIVITY')]


KeyboardInterrupt: 

In [6]:
len(step_semantics_result)

85

In [7]:
import pandas as pd
import psycopg2

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    database="cr",
    user="postgres",
    password="postgres"
)

# Execute an SQL query and create a dataframe 
muse_data_step = pd.read_sql("SELECT * FROM synthetic_data_version_1", conn).iloc[1:]
# muse_data_step.drop(muse_data_step.index[0])
muse_data_step.columns = muse_data_step.columns.str.upper()

conn.close()
display(muse_data_step)

,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
1,9,2,4,None,12,1.0,4,1,1.0,19.0,2,4.0
2,1,1,4,126359.50607515374,18,1.0,1,1,1.0,19.0,2,4.0
3,1,1,4,75127.85895569899,47,8.0,4,1,1.0,16.0,2,4.0
4,1,2,3,None,9,None,1,1,1.0,19.0,2,4.0
5,9,1,4,10790.54754111301,34,None,1,1,1.0,22.0,2,4.0
6,4,2,3,35818.3718385832,12,None,1,1,None,9.0,2,4.0
7,1,1,3,67608.02336629394,36,1.0,1,1,None,14.0,2,2.0
8,1,1,3,None,37,None,1,1,6.0,20.0,2,2.0
9,1,1,1,9828.667856309301,12,1.0,1,1,6.0,18.0,2,4.0
10,9,1,2,137591.90804369393,42,1.0,1,2,1.0,4.0,2,4.0


In [8]:
def make_col_int(df, *args):
    for col in args:
        df[col] = df[col].astype(float)
        df[col] = df[col].astype(int)
def make_col_string(df, *args):
    for col in args:
        df[col] = df[col].astype(str)
        
make_col_int(muse_data_step, 'RAC1P', 'SEX', 'REGION', 'ST', 'CIT', 'NATIVITY', 'DIS')
make_col_string(muse_data_step, 'PINCP', 'COW', 'MSP', 'SCHL', 'MIL')
print(muse_data_step.dtypes)

display(muse_data_step)

RAC1P        int64
SEX          int64
REGION       int64
PINCP       object
ST           int64
COW         object
CIT          int64
NATIVITY     int64
MSP         object
SCHL        object
DIS          int64
MIL         object
dtype: object


,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
1,9,2,4,None,12,1.0,4,1,1.0,19.0,2,4.0
2,1,1,4,126359.50607515374,18,1.0,1,1,1.0,19.0,2,4.0
3,1,1,4,75127.85895569899,47,8.0,4,1,1.0,16.0,2,4.0
4,1,2,3,None,9,None,1,1,1.0,19.0,2,4.0
5,9,1,4,10790.54754111301,34,None,1,1,1.0,22.0,2,4.0
6,4,2,3,35818.3718385832,12,None,1,1,None,9.0,2,4.0
7,1,1,3,67608.02336629394,36,1.0,1,1,None,14.0,2,2.0
8,1,1,3,None,37,None,1,1,6.0,20.0,2,2.0
9,1,1,1,9828.667856309301,12,1.0,1,1,6.0,18.0,2,4.0
10,9,1,2,137591.90804369393,42,1.0,1,2,1.0,4.0,2,4.0


In [9]:
tuples = [tup[1] for tup in step_semantics_result]
for t in tuples:
    row_index = muse_data_step.index[(muse_data_step == t).all(axis=1)]
    print(row_index)
    muse_data_step = muse_data_step.drop(row_index)

Int64Index([1748], dtype='int64')
Int64Index([732], dtype='int64')
Int64Index([1470], dtype='int64')
Int64Index([377], dtype='int64')
Int64Index([531], dtype='int64')
Int64Index([813], dtype='int64')
Int64Index([254], dtype='int64')
Int64Index([453], dtype='int64')
Int64Index([1986], dtype='int64')
Int64Index([977], dtype='int64')
Int64Index([1358], dtype='int64')
Int64Index([1506], dtype='int64')
Int64Index([230], dtype='int64')
Int64Index([778], dtype='int64')
Int64Index([1812], dtype='int64')
Int64Index([1215], dtype='int64')
Int64Index([1336], dtype='int64')
Int64Index([1242], dtype='int64')
Int64Index([1121], dtype='int64')
Int64Index([1653], dtype='int64')
Int64Index([898], dtype='int64')
Int64Index([1924], dtype='int64')
Int64Index([525], dtype='int64')
Int64Index([1069], dtype='int64')
Int64Index([2011], dtype='int64')
Int64Index([1887], dtype='int64')
Int64Index([994], dtype='int64')
Int64Index([630], dtype='int64')
Int64Index([577], dtype='int64')
Int64Index([555], dtype='int

In [10]:
display(muse_data_step)

,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
1,9,2,4,None,12,1.0,4,1,1.0,19.0,2,4.0
2,1,1,4,126359.50607515374,18,1.0,1,1,1.0,19.0,2,4.0
3,1,1,4,75127.85895569899,47,8.0,4,1,1.0,16.0,2,4.0
4,1,2,3,None,9,None,1,1,1.0,19.0,2,4.0
5,9,1,4,10790.54754111301,34,None,1,1,1.0,22.0,2,4.0
6,4,2,3,35818.3718385832,12,None,1,1,None,9.0,2,4.0
7,1,1,3,67608.02336629394,36,1.0,1,1,None,14.0,2,2.0
8,1,1,3,None,37,None,1,1,6.0,20.0,2,2.0
9,1,1,1,9828.667856309301,12,1.0,1,1,6.0,18.0,2,4.0
11,1,2,9,71588.23433336147,6,8.0,1,1,6.0,17.0,2,4.0


In [32]:
input_string = """
Int64Index([539], dtype='int64')
Int64Index([1652], dtype='int64')
Int64Index([1796], dtype='int64')
Int64Index([2038], dtype='int64')
Int64Index([570], dtype='int64')
Int64Index([1313], dtype='int64')
Int64Index([1088], dtype='int64')
Int64Index([927], dtype='int64')
Int64Index([1690], dtype='int64')
Int64Index([1510], dtype='int64')
Int64Index([1773], dtype='int64')
Int64Index([710], dtype='int64')
Int64Index([1145], dtype='int64')
Int64Index([422], dtype='int64')
Int64Index([1083], dtype='int64')
Int64Index([303], dtype='int64')
Int64Index([928], dtype='int64')
Int64Index([1169], dtype='int64')
Int64Index([362], dtype='int64')
Int64Index([382], dtype='int64')
Int64Index([814], dtype='int64')
Int64Index([1183], dtype='int64')
Int64Index([625], dtype='int64')
Int64Index([2029], dtype='int64')
Int64Index([1703], dtype='int64')
Int64Index([854], dtype='int64')
Int64Index([1357], dtype='int64')
Int64Index([1123], dtype='int64')
Int64Index([544], dtype='int64')
Int64Index([747], dtype='int64')
Int64Index([807], dtype='int64')
Int64Index([205], dtype='int64')
Int64Index([657], dtype='int64')
Int64Index([891], dtype='int64')
Int64Index([690], dtype='int64')
Int64Index([1760], dtype='int64')
Int64Index([171], dtype='int64')
Int64Index([1780], dtype='int64')
Int64Index([1292], dtype='int64')
Int64Index([488], dtype='int64')
Int64Index([277], dtype='int64')
Int64Index([1280], dtype='int64')
Int64Index([1934], dtype='int64')
Int64Index([1781], dtype='int64')
Int64Index([795], dtype='int64')
Int64Index([521], dtype='int64')
Int64Index([1722], dtype='int64')
Int64Index([1764], dtype='int64')
Int64Index([338], dtype='int64')
Int64Index([1110], dtype='int64')
Int64Index([482], dtype='int64')
Int64Index([564], dtype='int64')
Int64Index([813], dtype='int64')
Int64Index([1019], dtype='int64')
Int64Index([1465], dtype='int64')
Int64Index([1825], dtype='int64')
Int64Index([1642], dtype='int64')
Int64Index([1950], dtype='int64')
Int64Index([857], dtype='int64')
Int64Index([291], dtype='int64')
Int64Index([1929], dtype='int64')
Int64Index([1681], dtype='int64')
Int64Index([1036], dtype='int64')
Int64Index([1622], dtype='int64')
Int64Index([74], dtype='int64')
Int64Index([1241], dtype='int64')
Int64Index([1772], dtype='int64')
Int64Index([806], dtype='int64')
Int64Index([1924], dtype='int64')
Int64Index([1263], dtype='int64')
Int64Index([1247], dtype='int64')
Int64Index([705], dtype='int64')
Int64Index([200], dtype='int64')
Int64Index([436], dtype='int64')
Int64Index([339], dtype='int64')
Int64Index([1749], dtype='int64')
Int64Index([1003], dtype='int64')
Int64Index([1256], dtype='int64')
Int64Index([1979], dtype='int64')
Int64Index([1697], dtype='int64')
Int64Index([1223], dtype='int64')
Int64Index([328], dtype='int64')
Int64Index([260], dtype='int64')
Int64Index([1396], dtype='int64')
Int64Index([587], dtype='int64')
Int64Index([1915], dtype='int64')
Int64Index([979], dtype='int64')
Int64Index([354], dtype='int64')
Int64Index([1521], dtype='int64')
Int64Index([1517], dtype='int64')
Int64Index([1273], dtype='int64')
Int64Index([332], dtype='int64')
Int64Index([439], dtype='int64')
Int64Index([288], dtype='int64')
Int64Index([383], dtype='int64')
Int64Index([1430], dtype='int64')
Int64Index([941], dtype='int64')
Int64Index([1098], dtype='int64')
Int64Index([1926], dtype='int64')
Int64Index([1972], dtype='int64')
Int64Index([442], dtype='int64')
Int64Index([1418], dtype='int64')
Int64Index([446], dtype='int64')
Int64Index([1235], dtype='int64')
Int64Index([1593], dtype='int64')
Int64Index([289], dtype='int64')
Int64Index([1956], dtype='int64')
Int64Index([1502], dtype='int64')
Int64Index([825], dtype='int64')
Int64Index([24], dtype='int64')
Int64Index([1646], dtype='int64')
Int64Index([2080], dtype='int64')
Int64Index([1009], dtype='int64')
Int64Index([819], dtype='int64')
Int64Index([766], dtype='int64')
Int64Index([1230], dtype='int64')
Int64Index([1102], dtype='int64')
Int64Index([336], dtype='int64')
Int64Index([568], dtype='int64')
Int64Index([35], dtype='int64')
Int64Index([1326], dtype='int64')
Int64Index([950], dtype='int64')
Int64Index([1130], dtype='int64')
Int64Index([1536], dtype='int64')
Int64Index([5], dtype='int64')
Int64Index([410], dtype='int64')
Int64Index([1253], dtype='int64')
Int64Index([359], dtype='int64')
Int64Index([676], dtype='int64')
Int64Index([982], dtype='int64')
Int64Index([1813], dtype='int64')
Int64Index([622], dtype='int64')
Int64Index([1165], dtype='int64')
Int64Index([1596], dtype='int64')
Int64Index([892], dtype='int64')
Int64Index([249], dtype='int64')
Int64Index([1921], dtype='int64')
Int64Index([1826], dtype='int64')
Int64Index([40], dtype='int64')
Int64Index([1392], dtype='int64')
Int64Index([1958], dtype='int64')
Int64Index([244], dtype='int64')
Int64Index([1117], dtype='int64')
Int64Index([10], dtype='int64')
Int64Index([1893], dtype='int64')
Int64Index([460], dtype='int64')
Int64Index([534], dtype='int64')
Int64Index([2018], dtype='int64')
Int64Index([1761], dtype='int64')
Int64Index([1748], dtype='int64')
Int64Index([792], dtype='int64')
Int64Index([538], dtype='int64')
Int64Index([885], dtype='int64')
Int64Index([719], dtype='int64')
Int64Index([267], dtype='int64')
Int64Index([1908], dtype='int64')
Int64Index([1281], dtype='int64')
Int64Index([1719], dtype='int64')
Int64Index([363], dtype='int64')
Int64Index([637], dtype='int64')
Int64Index([1643], dtype='int64')
Int64Index([1701], dtype='int64')
Int64Index([1408], dtype='int64')
Int64Index([1135], dtype='int64')
Int64Index([843], dtype='int64')
Int64Index([1425], dtype='int64')
Int64Index([610], dtype='int64')
Int64Index([1243], dtype='int64')
Int64Index([609], dtype='int64')
Int64Index([677], dtype='int64')
Int64Index([574], dtype='int64')
Int64Index([1497], dtype='int64')
Int64Index([1080], dtype='int64')
Int64Index([507], dtype='int64')
Int64Index([489], dtype='int64')
Int64Index([1197], dtype='int64')
Int64Index([1092], dtype='int64')
Int64Index([1269], dtype='int64')
Int64Index([696], dtype='int64')
Int64Index([1731], dtype='int64')
Int64Index([1507], dtype='int64')
Int64Index([313], dtype='int64')
Int64Index([779], dtype='int64')
Int64Index([1210], dtype='int64')
Int64Index([52], dtype='int64')
Int64Index([649], dtype='int64')
Int64Index([1477], dtype='int64')
Int64Index([1366], dtype='int64')
Int64Index([1449], dtype='int64')
Int64Index([1996], dtype='int64')
Int64Index([2094], dtype='int64')
Int64Index([862], dtype='int64')
Int64Index([1559], dtype='int64')
Int64Index([1608], dtype='int64')
Int64Index([1568], dtype='int64')
Int64Index([166], dtype='int64')
Int64Index([463], dtype='int64')
Int64Index([486], dtype='int64')
Int64Index([1989], dtype='int64')
Int64Index([1372], dtype='int64')
Int64Index([1410], dtype='int64')
Int64Index([666], dtype='int64')
Int64Index([114], dtype='int64')
Int64Index([1886], dtype='int64')
Int64Index([919], dtype='int64')
Int64Index([1969], dtype='int64')
Int64Index([1804], dtype='int64')
Int64Index([693], dtype='int64')
Int64Index([1378], dtype='int64')
Int64Index([2014], dtype='int64')
Int64Index([319], dtype='int64')
Int64Index([1056], dtype='int64')
Int64Index([514], dtype='int64')
Int64Index([1659], dtype='int64')
Int64Index([268], dtype='int64')
Int64Index([1411], dtype='int64')
Int64Index([1191], dtype='int64')
Int64Index([70], dtype='int64')
Int64Index([207], dtype='int64')
Int64Index([1694], dtype='int64')
Int64Index([51], dtype='int64')
Int64Index([920], dtype='int64')
Int64Index([1573], dtype='int64')
Int64Index([591], dtype='int64')
Int64Index([599], dtype='int64')
Int64Index([2086], dtype='int64')
Int64Index([1290], dtype='int64')
Int64Index([1953], dtype='int64')
Int64Index([1351], dtype='int64')
Int64Index([82], dtype='int64')
Int64Index([9], dtype='int64')
Int64Index([644], dtype='int64')
Int64Index([575], dtype='int64')
Int64Index([683], dtype='int64')
Int64Index([1352], dtype='int64')
Int64Index([1490], dtype='int64')
Int64Index([1409], dtype='int64')
Int64Index([1997], dtype='int64')
Int64Index([908], dtype='int64')
Int64Index([1297], dtype='int64')
Int64Index([1607], dtype='int64')
Int64Index([242], dtype='int64')
Int64Index([812], dtype='int64')
Int64Index([1328], dtype='int64')
Int64Index([1208], dtype='int64')
Int64Index([184], dtype='int64')
Int64Index([373], dtype='int64')
Int64Index([161], dtype='int64')
Int64Index([1708], dtype='int64')
Int64Index([1842], dtype='int64')
Int64Index([433], dtype='int64')
Int64Index([1157], dtype='int64')
Int64Index([1734], dtype='int64')
Int64Index([1398], dtype='int64')
Int64Index([1150], dtype='int64')
Int64Index([588], dtype='int64')
Int64Index([203], dtype='int64')
Int64Index([578], dtype='int64')
Int64Index([1586], dtype='int64')
Int64Index([893], dtype='int64')
Int64Index([793], dtype='int64')
Int64Index([281], dtype='int64')
Int64Index([102], dtype='int64')
Int64Index([673], dtype='int64')
Int64Index([1820], dtype='int64')
Int64Index([910], dtype='int64')
Int64Index([1987], dtype='int64')
Int64Index([1706], dtype='int64')
Int64Index([1478], dtype='int64')
Int64Index([1030], dtype='int64')
Int64Index([741], dtype='int64')
Int64Index([43], dtype='int64')
Int64Index([1375], dtype='int64')
Int64Index([183], dtype='int64')
Int64Index([594], dtype='int64')
Int64Index([282], dtype='int64')
Int64Index([1323], dtype='int64')
Int64Index([1116], dtype='int64')
Int64Index([985], dtype='int64')
Int64Index([1567], dtype='int64')
Int64Index([133], dtype='int64')
Int64Index([1558], dtype='int64')
Int64Index([1331], dtype='int64')
Int64Index([698], dtype='int64')
Int64Index([257], dtype='int64')
Int64Index([1205], dtype='int64')
Int64Index([430], dtype='int64')
Int64Index([1560], dtype='int64')
Int64Index([384], dtype='int64')
Int64Index([1335], dtype='int64')
Int64Index([501], dtype='int64')
Int64Index([490], dtype='int64')
Int64Index([936], dtype='int64')
Int64Index([483], dtype='int64')
Int64Index([1654], dtype='int64')
Int64Index([1128], dtype='int64')
Int64Index([540], dtype='int64')
Int64Index([858], dtype='int64')
Int64Index([1356], dtype='int64')
Int64Index([1277], dtype='int64')
Int64Index([1581], dtype='int64')
Int64Index([317], dtype='int64')
Int64Index([477], dtype='int64')
Int64Index([527], dtype='int64')
Int64Index([1687], dtype='int64')
Int64Index([641], dtype='int64')
Int64Index([283], dtype='int64')
Int64Index([537], dtype='int64')
Int64Index([265], dtype='int64')
Int64Index([2020], dtype='int64')
Int64Index([1525], dtype='int64')
Int64Index([1002], dtype='int64')
Int64Index([1086], dtype='int64')
Int64Index([2069], dtype='int64')
Int64Index([1937], dtype='int64')
Int64Index([1437], dtype='int64')
Int64Index([1018], dtype='int64')
Int64Index([1294], dtype='int64')
Int64Index([1350], dtype='int64')
Int64Index([1955], dtype='int64')
Int64Index([1259], dtype='int64')
Int64Index([304], dtype='int64')
Int64Index([2027], dtype='int64')
Int64Index([1611], dtype='int64')
Int64Index([311], dtype='int64')
Int64Index([2087], dtype='int64')
Int64Index([326], dtype='int64')
Int64Index([1161], dtype='int64')
Int64Index([1118], dtype='int64')
Int64Index([1544], dtype='int64')
Int64Index([1442], dtype='int64')
Int64Index([662], dtype='int64')
Int64Index([583], dtype='int64')
Int64Index([604], dtype='int64')
Int64Index([1173], dtype='int64')
Int64Index([1270], dtype='int64')
Int64Index([1412], dtype='int64')
Int64Index([1085], dtype='int64')
Int64Index([1878], dtype='int64')
Int64Index([1806], dtype='int64')
Int64Index([671], dtype='int64')
Int64Index([492], dtype='int64')
Int64Index([1252], dtype='int64')
Int64Index([1199], dtype='int64')
Int64Index([375], dtype='int64')
Int64Index([1752], dtype='int64')
Int64Index([1461], dtype='int64')
Int64Index([543], dtype='int64')
Int64Index([225], dtype='int64')
Int64Index([1447], dtype='int64')
Int64Index([222], dtype='int64')
Int64Index([1194], dtype='int64')
Int64Index([776], dtype='int64')
Int64Index([65], dtype='int64')
Int64Index([411], dtype='int64')
Int64Index([1868], dtype='int64')
Int64Index([131], dtype='int64')
Int64Index([685], dtype='int64')
Int64Index([723], dtype='int64')
Int64Index([1158], dtype='int64')
Int64Index([754], dtype='int64')
Int64Index([1920], dtype='int64')
Int64Index([547], dtype='int64')
Int64Index([912], dtype='int64')
Int64Index([1837], dtype='int64')
Int64Index([1580], dtype='int64')
Int64Index([195], dtype='int64')
Int64Index([1818], dtype='int64')
Int64Index([1070], dtype='int64')
Int64Index([58], dtype='int64')
Int64Index([1494], dtype='int64')
Int64Index([600], dtype='int64')
Int64Index([1000], dtype='int64')
Int64Index([139], dtype='int64')
Int64Index([471], dtype='int64')
Int64Index([787], dtype='int64')
Int64Index([1214], dtype='int64')
Int64Index([1228], dtype='int64')
Int64Index([1740], dtype='int64')
Int64Index([1863], dtype='int64')
Int64Index([512], dtype='int64')
Int64Index([1143], dtype='int64')
Int64Index([211], dtype='int64')
Int64Index([1200], dtype='int64')
Int64Index([1672], dtype='int64')
Int64Index([330], dtype='int64')
Int64Index([1082], dtype='int64')
Int64Index([1216], dtype='int64')
Int64Index([1554], dtype='int64')
Int64Index([1343], dtype='int64')
Int64Index([1084], dtype='int64')
Int64Index([887], dtype='int64')
Int64Index([642], dtype='int64')
Int64Index([218], dtype='int64')
Int64Index([859], dtype='int64')
Int64Index([1555], dtype='int64')
Int64Index([1178], dtype='int64')
Int64Index([1910], dtype='int64')
Int64Index([1132], dtype='int64')
Int64Index([785], dtype='int64')
Int64Index([500], dtype='int64')
Int64Index([44], dtype='int64')
Int64Index([1443], dtype='int64')
Int64Index([557], dtype='int64')
Int64Index([1393], dtype='int64')
Int64Index([689], dtype='int64')
Int64Index([598], dtype='int64')
Int64Index([451], dtype='int64')
Int64Index([269], dtype='int64')
Int64Index([652], dtype='int64')
Int64Index([154], dtype='int64')
Int64Index([1711], dtype='int64')
Int64Index([95], dtype='int64')
Int64Index([1601], dtype='int64')
Int64Index([1674], dtype='int64')
Int64Index([1765], dtype='int64')
Int64Index([418], dtype='int64')
Int64Index([768], dtype='int64')
Int64Index([1192], dtype='int64')
Int64Index([1415], dtype='int64')
Int64Index([616], dtype='int64')
Int64Index([551], dtype='int64')
Int64Index([870], dtype='int64')
Int64Index([899], dtype='int64')
Int64Index([706], dtype='int64')
Int64Index([297], dtype='int64')
Int64Index([143], dtype='int64')
Int64Index([1965], dtype='int64')
Int64Index([780], dtype='int64')
Int64Index([1051], dtype='int64')
Int64Index([1193], dtype='int64')
Int64Index([934], dtype='int64')
Int64Index([416], dtype='int64')
Int64Index([614], dtype='int64')
Int64Index([1033], dtype='int64')
Int64Index([1404], dtype='int64')
Int64Index([1981], dtype='int64')
Int64Index([615], dtype='int64')
Int64Index([1680], dtype='int64')
Int64Index([1468], dtype='int64')
Int64Index([821], dtype='int64')
Int64Index([536], dtype='int64')
Int64Index([325], dtype='int64')
Int64Index([603], dtype='int64')
Int64Index([461], dtype='int64')
Int64Index([125], dtype='int64')
Int64Index([2075], dtype='int64')
Int64Index([935], dtype='int64')
Int64Index([2066], dtype='int64')
Int64Index([909], dtype='int64')
Int64Index([1303], dtype='int64')
Int64Index([1889], dtype='int64')
Int64Index([235], dtype='int64')
Int64Index([815], dtype='int64')
Int64Index([1258], dtype='int64')
Int64Index([1501], dtype='int64')
Int64Index([83], dtype='int64')
Int64Index([664], dtype='int64')
Int64Index([846], dtype='int64')
Int64Index([358], dtype='int64')
Int64Index([232], dtype='int64')
Int64Index([192], dtype='int64')
Int64Index([665], dtype='int64')
Int64Index([264], dtype='int64')
Int64Index([1283], dtype='int64')
Int64Index([1151], dtype='int64')
Int64Index([465], dtype='int64')
Int64Index([1729], dtype='int64')
Int64Index([1010], dtype='int64')
Int64Index([1472], dtype='int64')
Int64Index([1329], dtype='int64')
Int64Index([2077], dtype='int64')
Int64Index([1272], dtype='int64')
Int64Index([1859], dtype='int64')
Int64Index([1103], dtype='int64')
Int64Index([1776], dtype='int64')
Int64Index([144], dtype='int64')
Int64Index([1016], dtype='int64')
Int64Index([560], dtype='int64')
Int64Index([186], dtype='int64')
Int64Index([210], dtype='int64')
Int64Index([1890], dtype='int64')
Int64Index([630], dtype='int64')
Int64Index([369], dtype='int64')
Int64Index([315], dtype='int64')
Int64Index([2045], dtype='int64')
Int64Index([157], dtype='int64')
Int64Index([1141], dtype='int64')
Int64Index([234], dtype='int64')
Int64Index([1894], dtype='int64')
Int64Index([1254], dtype='int64')
Int64Index([804], dtype='int64')
Int64Index([1561], dtype='int64')
Int64Index([1617], dtype='int64')
Int64Index([1839], dtype='int64')
Int64Index([1295], dtype='int64')
Int64Index([1159], dtype='int64')
Int64Index([1155], dtype='int64')
Int64Index([1571], dtype='int64')
Int64Index([1097], dtype='int64')
Int64Index([227], dtype='int64')
Int64Index([216], dtype='int64')
Int64Index([1301], dtype='int64')
Int64Index([314], dtype='int64')
Int64Index([1094], dtype='int64')
Int64Index([2089], dtype='int64')
Int64Index([2095], dtype='int64')
Int64Index([850], dtype='int64')
Int64Index([481], dtype='int64')
Int64Index([1759], dtype='int64')
Int64Index([323], dtype='int64')
Int64Index([875], dtype='int64')
Int64Index([947], dtype='int64')
Int64Index([1293], dtype='int64')
Int64Index([1394], dtype='int64')
Int64Index([758], dtype='int64')
Int64Index([2072], dtype='int64')
Int64Index([1201], dtype='int64')
Int64Index([1870], dtype='int64')
Int64Index([447], dtype='int64')
Int64Index([450], dtype='int64')
Int64Index([443], dtype='int64')
Int64Index([1365], dtype='int64')
Int64Index([749], dtype='int64')
Int64Index([2061], dtype='int64')
Int64Index([1911], dtype='int64')
Int64Index([735], dtype='int64')
Int64Index([352], dtype='int64')
Int64Index([946], dtype='int64')
Int64Index([1707], dtype='int64')
Int64Index([2048], dtype='int64')
Int64Index([1318], dtype='int64')
Int64Index([2092], dtype='int64')
Int64Index([196], dtype='int64')
Int64Index([1131], dtype='int64')
Int64Index([1990], dtype='int64')
Int64Index([1440], dtype='int64')
Int64Index([680], dtype='int64')
Int64Index([906], dtype='int64')
Int64Index([399], dtype='int64')
Int64Index([1421], dtype='int64')
Int64Index([635], dtype='int64')
Int64Index([923], dtype='int64')
Int64Index([969], dtype='int64')
Int64Index([1812], dtype='int64')
Int64Index([1725], dtype='int64')
Int64Index([1089], dtype='int64')
Int64Index([590], dtype='int64')
Int64Index([1556], dtype='int64')
Int64Index([1922], dtype='int64')
Int64Index([1203], dtype='int64')
Int64Index([1791], dtype='int64')
Int64Index([1845], dtype='int64')
Int64Index([468], dtype='int64')
Int64Index([1429], dtype='int64')
Int64Index([611], dtype='int64')
Int64Index([1819], dtype='int64')
Int64Index([1831], dtype='int64')
Int64Index([577], dtype='int64')
Int64Index([1134], dtype='int64')
Int64Index([822], dtype='int64')
Int64Index([1951], dtype='int64')
Int64Index([876], dtype='int64')
Int64Index([1721], dtype='int64')
Int64Index([506], dtype='int64')
Int64Index([580], dtype='int64')
Int64Index([1028], dtype='int64')
Int64Index([1058], dtype='int64')
Int64Index([1384], dtype='int64')
Int64Index([2006], dtype='int64')
Int64Index([639], dtype='int64')
Int64Index([1042], dtype='int64')
Int64Index([890], dtype='int64')
Int64Index([428], dtype='int64')
Int64Index([385], dtype='int64')
Int64Index([2098], dtype='int64')
Int64Index([245], dtype='int64')
Int64Index([2041], dtype='int64')
Int64Index([466], dtype='int64')
Int64Index([1794], dtype='int64')
Int64Index([532], dtype='int64')
Int64Index([1483], dtype='int64')
Int64Index([1762], dtype='int64')
Int64Index([190], dtype='int64')
Int64Index([1007], dtype='int64')
Int64Index([1268], dtype='int64')
Int64Index([1154], dtype='int64')
Int64Index([2070], dtype='int64')
Int64Index([365], dtype='int64')
Int64Index([2052], dtype='int64')
Int64Index([631], dtype='int64')
Int64Index([573], dtype='int64')
Int64Index([215], dtype='int64')
Int64Index([11], dtype='int64')
Int64Index([31], dtype='int64')
Int64Index([1319], dtype='int64')
Int64Index([1325], dtype='int64')
Int64Index([241], dtype='int64')
Int64Index([104], dtype='int64')
Int64Index([335], dtype='int64')
Int64Index([276], dtype='int64')
Int64Index([327], dtype='int64')
Int64Index([162], dtype='int64')
Int64Index([2005], dtype='int64')
Int64Index([2000], dtype='int64')
Int64Index([552], dtype='int64')
Int64Index([1342], dtype='int64')
Int64Index([1600], dtype='int64')
Int64Index([127], dtype='int64')
Int64Index([407], dtype='int64')
Int64Index([742], dtype='int64')
Int64Index([1362], dtype='int64')
Int64Index([903], dtype='int64')
Int64Index([1887], dtype='int64')
Int64Index([1286], dtype='int64')
Int64Index([1435], dtype='int64')
Int64Index([392], dtype='int64')
Int64Index([1385], dtype='int64')
Int64Index([922], dtype='int64')
Int64Index([669], dtype='int64')
Int64Index([888], dtype='int64')
Int64Index([880], dtype='int64')
Int64Index([480], dtype='int64')
Int64Index([286], dtype='int64')
Int64Index([986], dtype='int64')
Int64Index([293], dtype='int64')
Int64Index([728], dtype='int64')
Int64Index([92], dtype='int64')
Int64Index([1545], dtype='int64')
Int64Index([258], dtype='int64')
Int64Index([679], dtype='int64')
Int64Index([1905], dtype='int64')
Int64Index([1407], dtype='int64')
Int64Index([713], dtype='int64')
Int64Index([2068], dtype='int64')
Int64Index([105], dtype='int64')
Int64Index([515], dtype='int64')
Int64Index([847], dtype='int64')
Int64Index([85], dtype='int64')
Int64Index([470], dtype='int64')
Int64Index([1853], dtype='int64')
Int64Index([1487], dtype='int64')
Int64Index([271], dtype='int64')
Int64Index([1649], dtype='int64')
Int64Index([1397], dtype='int64')
Int64Index([561], dtype='int64')
Int64Index([939], dtype='int64')
Int64Index([1053], dtype='int64')
Int64Index([632], dtype='int64')
Int64Index([1543], dtype='int64')
Int64Index([111], dtype='int64')
Int64Index([2084], dtype='int64')
Int64Index([1883], dtype='int64')
Int64Index([1355], dtype='int64')
Int64Index([1957], dtype='int64')
Int64Index([686], dtype='int64')
Int64Index([1023], dtype='int64')
Int64Index([2022], dtype='int64')
Int64Index([371], dtype='int64')
Int64Index([1180], dtype='int64')
Int64Index([1220], dtype='int64')
Int64Index([756], dtype='int64')
Int64Index([1057], dtype='int64')
Int64Index([341], dtype='int64')
Int64Index([970], dtype='int64')
Int64Index([542], dtype='int64')
Int64Index([1852], dtype='int64')
Int64Index([1986], dtype='int64')
Int64Index([546], dtype='int64')
Int64Index([1866], dtype='int64')
Int64Index([142], dtype='int64')
Int64Index([1260], dtype='int64')
Int64Index([346], dtype='int64')
Int64Index([1], dtype='int64')
Int64Index([1550], dtype='int64')
Int64Index([266], dtype='int64')
Int64Index([206], dtype='int64')
Int64Index([1420], dtype='int64')
Int64Index([299], dtype='int64')
Int64Index([290], dtype='int64')
Int64Index([440], dtype='int64')
Int64Index([1167], dtype='int64')
Int64Index([1876], dtype='int64')
Int64Index([2053], dtype='int64')
Int64Index([270], dtype='int64')
Int64Index([628], dtype='int64')
Int64Index([1261], dtype='int64')
Int64Index([1149], dtype='int64')
Int64Index([419], dtype='int64')
Int64Index([478], dtype='int64')
Int64Index([285], dtype='int64')
Int64Index([301], dtype='int64')
Int64Index([397], dtype='int64')
Int64Index([1650], dtype='int64')
Int64Index([823], dtype='int64')
Int64Index([1702], dtype='int64')
Int64Index([1668], dtype='int64')
Int64Index([464], dtype='int64')
Int64Index([1612], dtype='int64')
Int64Index([467], dtype='int64')
Int64Index([1144], dtype='int64')
Int64Index([882], dtype='int64')
Int64Index([1816], dtype='int64')
Int64Index([855], dtype='int64')
Int64Index([1952], dtype='int64')
Int64Index([699], dtype='int64')
Int64Index([170], dtype='int64')
Int64Index([1904], dtype='int64')
Int64Index([1068], dtype='int64')
Int64Index([20], dtype='int64')
Int64Index([1413], dtype='int64')
Int64Index([28], dtype='int64')
Int64Index([798], dtype='int64')
Int64Index([1835], dtype='int64')
Int64Index([145], dtype='int64')
Int64Index([504], dtype='int64')
Int64Index([1619], dtype='int64')
Int64Index([474], dtype='int64')
Int64Index([658], dtype='int64')
Int64Index([312], dtype='int64')
Int64Index([992], dtype='int64')
Int64Index([158], dtype='int64')
Int64Index([425], dtype='int64')
Int64Index([1122], dtype='int64')
Int64Index([976], dtype='int64')
Int64Index([584], dtype='int64')
Int64Index([292], dtype='int64')
Int64Index([110], dtype='int64')
Int64Index([226], dtype='int64')
Int64Index([757], dtype='int64')
Int64Index([1963], dtype='int64')
Int64Index([26], dtype='int64')
Int64Index([1279], dtype='int64')
Int64Index([1321], dtype='int64')
Int64Index([944], dtype='int64')
Int64Index([1339], dtype='int64')
Int64Index([1484], dtype='int64')
Int64Index([1565], dtype='int64')
Int64Index([1744], dtype='int64')
Int64Index([734], dtype='int64')
Int64Index([606], dtype='int64')
Int64Index([1031], dtype='int64')
Int64Index([1621], dtype='int64')
Int64Index([427], dtype='int64')
Int64Index([1656], dtype='int64')
Int64Index([1539], dtype='int64')
Int64Index([1557], dtype='int64')
Int64Index([1267], dtype='int64')
Int64Index([975], dtype='int64')
Int64Index([2044], dtype='int64')
Int64Index([2050], dtype='int64')
Int64Index([178], dtype='int64')
Int64Index([1239], dtype='int64')
Int64Index([856], dtype='int64')
Int64Index([122], dtype='int64')
Int64Index([1382], dtype='int64')
Int64Index([973], dtype='int64')
Int64Index([2009], dtype='int64')
Int64Index([1750], dtype='int64')
Int64Index([1106], dtype='int64')
Int64Index([1533], dtype='int64')
Int64Index([1739], dtype='int64')
Int64Index([520], dtype='int64')
Int64Index([791], dtype='int64')
Int64Index([1244], dtype='int64')
Int64Index([1354], dtype='int64')
Int64Index([1456], dtype='int64')
Int64Index([307], dtype='int64')
Int64Index([1587], dtype='int64')
Int64Index([452], dtype='int64')
Int64Index([853], dtype='int64')
Int64Index([509], dtype='int64')
Int64Index([1660], dtype='int64')
Int64Index([101], dtype='int64')
Int64Index([1246], dtype='int64')
Int64Index([239], dtype='int64')
Int64Index([585], dtype='int64')
Int64Index([2073], dtype='int64')
Int64Index([1713], dtype='int64')
Int64Index([566], dtype='int64')
Int64Index([682], dtype='int64')
Int64Index([298], dtype='int64')
Int64Index([1495], dtype='int64')
Int64Index([1553], dtype='int64')
Int64Index([1316], dtype='int64')
Int64Index([1782], dtype='int64')
Int64Index([1162], dtype='int64')
Int64Index([1175], dtype='int64')
Int64Index([230], dtype='int64')
Int64Index([370], dtype='int64')
Int64Index([251], dtype='int64')
Int64Index([1400], dtype='int64')
Int64Index([1078], dtype='int64')
Int64Index([21], dtype='int64')
Int64Index([746], dtype='int64')
Int64Index([494], dtype='int64')
Int64Index([1196], dtype='int64')
Int64Index([864], dtype='int64')
Int64Index([1821], dtype='int64')
Int64Index([2062], dtype='int64')
Int64Index([36], dtype='int64')
Int64Index([553], dtype='int64')
Int64Index([189], dtype='int64')
Int64Index([1189], dtype='int64')
Int64Index([2010], dtype='int64')
Int64Index([1112], dtype='int64')
Int64Index([655], dtype='int64')
Int64Index([1726], dtype='int64')
Int64Index([1265], dtype='int64')
Int64Index([2063], dtype='int64')
Int64Index([829], dtype='int64')
Int64Index([722], dtype='int64')
Int64Index([34], dtype='int64')
Int64Index([487], dtype='int64')
Int64Index([420], dtype='int64')
Int64Index([435], dtype='int64')
Int64Index([1077], dtype='int64')
Int64Index([1566], dtype='int64')
Int64Index([2081], dtype='int64')
Int64Index([1606], dtype='int64')
Int64Index([1109], dtype='int64')
Int64Index([1576], dtype='int64')
Int64Index([796], dtype='int64')
Int64Index([963], dtype='int64')
Int64Index([498], dtype='int64')
Int64Index([711], dtype='int64')
Int64Index([130], dtype='int64')
Int64Index([1181], dtype='int64')
Int64Index([730], dtype='int64')
Int64Index([1712], dtype='int64')
Int64Index([1653], dtype='int64')
Int64Index([1741], dtype='int64')
Int64Index([991], dtype='int64')
Int64Index([1658], dtype='int64')
Int64Index([826], dtype='int64')
Int64Index([502], dtype='int64')
Int64Index([350], dtype='int64')
Int64Index([56], dtype='int64')
Int64Index([1498], dtype='int64')
Int64Index([1809], dtype='int64')
Int64Index([582], dtype='int64')
Int64Index([405], dtype='int64')
Int64Index([1026], dtype='int64')
Int64Index([1063], dtype='int64')
Int64Index([98], dtype='int64')
Int64Index([1234], dtype='int64')
Int64Index([1137], dtype='int64')
Int64Index([1245], dtype='int64')
Int64Index([1206], dtype='int64')
Int64Index([659], dtype='int64')
Int64Index([380], dtype='int64')
Int64Index([1376], dtype='int64')
Int64Index([126], dtype='int64')
Int64Index([907], dtype='int64')
Int64Index([1190], dtype='int64')
Int64Index([212], dtype='int64')
Int64Index([1562], dtype='int64')
Int64Index([308], dtype='int64')
Int64Index([901], dtype='int64')
Int64Index([714], dtype='int64')
Int64Index([22], dtype='int64')
Int64Index([1219], dtype='int64')
Int64Index([839], dtype='int64')
Int64Index([1133], dtype='int64')
Int64Index([1784], dtype='int64')
Int64Index([445], dtype='int64')
Int64Index([236], dtype='int64')
Int64Index([917], dtype='int64')
Int64Index([841], dtype='int64')
Int64Index([1939], dtype='int64')
Int64Index([1696], dtype='int64')
Int64Index([781], dtype='int64')
Int64Index([1466], dtype='int64')
Int64Index([1579], dtype='int64')
Int64Index([1846], dtype='int64')
Int64Index([1899], dtype='int64')
Int64Index([167], dtype='int64')
Int64Index([1705], dtype='int64')
Int64Index([762], dtype='int64')
Int64Index([71], dtype='int64')
Int64Index([1207], dtype='int64')
Int64Index([1857], dtype='int64')
Int64Index([17], dtype='int64')
Int64Index([736], dtype='int64')
Int64Index([1276], dtype='int64')
Int64Index([1609], dtype='int64')
Int64Index([1438], dtype='int64')
Int64Index([63], dtype='int64')
Int64Index([2049], dtype='int64')
Int64Index([549], dtype='int64')
Int64Index([176], dtype='int64')
Int64Index([1640], dtype='int64')
Int64Index([4], dtype='int64')
Int64Index([454], dtype='int64')
Int64Index([1945], dtype='int64')
Int64Index([629], dtype='int64')
Int64Index([155], dtype='int64')
Int64Index([2091], dtype='int64')
Int64Index([678], dtype='int64')
Int64Index([1224], dtype='int64')
Int64Index([1742], dtype='int64')
Int64Index([456], dtype='int64')
Int64Index([1540], dtype='int64')
Int64Index([1032], dtype='int64')
Int64Index([1436], dtype='int64')
Int64Index([874], dtype='int64')
Int64Index([1795], dtype='int64')
Int64Index([201], dtype='int64')
Int64Index([1314], dtype='int64')
Int64Index([209], dtype='int64')
Int64Index([1524], dtype='int64')
Int64Index([1108], dtype='int64')
Int64Index([305], dtype='int64')
Int64Index([1745], dtype='int64')
Int64Index([1059], dtype='int64')
Int64Index([1991], dtype='int64')
Int64Index([434], dtype='int64')
Int64Index([1971], dtype='int64')
Int64Index([1358], dtype='int64')
Int64Index([1902], dtype='int64')
Int64Index([636], dtype='int64')
Int64Index([646], dtype='int64')
Int64Index([943], dtype='int64')
Int64Index([1079], dtype='int64')
Int64Index([458], dtype='int64')
Int64Index([1733], dtype='int64')
Int64Index([1743], dtype='int64')
Int64Index([1309], dtype='int64')
Int64Index([1709], dtype='int64')
Int64Index([175], dtype='int64')
Int64Index([1513], dtype='int64')
Int64Index([810], dtype='int64')
Int64Index([786], dtype='int64')
Int64Index([1638], dtype='int64')
Int64Index([1959], dtype='int64')
Int64Index([863], dtype='int64')
Int64Index([743], dtype='int64')
Int64Index([208], dtype='int64')
Int64Index([1334], dtype='int64')
Int64Index([499], dtype='int64')
Int64Index([1014], dtype='int64')
Int64Index([99], dtype='int64')
Int64Index([1854], dtype='int64')
Int64Index([777], dtype='int64')
Int64Index([643], dtype='int64')
Int64Index([1238], dtype='int64')
Int64Index([89], dtype='int64')
Int64Index([1076], dtype='int64')
Int64Index([1476], dtype='int64')
Int64Index([1855], dtype='int64')
Int64Index([223], dtype='int64')
Int64Index([135], dtype='int64')
Int64Index([681], dtype='int64')
Int64Index([744], dtype='int64')
Int64Index([595], dtype='int64')
Int64Index([348], dtype='int64')
Int64Index([1176], dtype='int64')
Int64Index([884], dtype='int64')
Int64Index([1667], dtype='int64')
Int64Index([607], dtype='int64')
Int64Index([1970], dtype='int64')
Int64Index([457], dtype='int64')
Int64Index([1417], dtype='int64')
Int64Index([322], dtype='int64')
Int64Index([1832], dtype='int64')
Int64Index([86], dtype='int64')
Int64Index([794], dtype='int64')
Int64Index([1444], dtype='int64')
Int64Index([408], dtype='int64')
Int64Index([708], dtype='int64')
Int64Index([193], dtype='int64')
Int64Index([93], dtype='int64')
Int64Index([1227], dtype='int64')
Int64Index([152], dtype='int64')
Int64Index([19], dtype='int64')
Int64Index([872], dtype='int64')
Int64Index([716], dtype='int64')
Int64Index([727], dtype='int64')
Int64Index([978], dtype='int64')
Int64Index([1209], dtype='int64')
Int64Index([670], dtype='int64')
Int64Index([1008], dtype='int64')
Int64Index([1067], dtype='int64')
Int64Index([497], dtype='int64')
Int64Index([1797], dtype='int64')
Int64Index([1925], dtype='int64')
Int64Index([933], dtype='int64')
Int64Index([394], dtype='int64')
Int64Index([1805], dtype='int64')
Int64Index([2067], dtype='int64')
Int64Index([883], dtype='int64')
Int64Index([988], dtype='int64')
Int64Index([554], dtype='int64')
Int64Index([740], dtype='int64')
Int64Index([2043], dtype='int64')
Int64Index([1485], dtype='int64')
Int64Index([956], dtype='int64')
Int64Index([2056], dtype='int64')
Int64Index([1923], dtype='int64')
Int64Index([256], dtype='int64')
Int64Index([712], dtype='int64')
Int64Index([1605], dtype='int64')
Int64Index([372], dtype='int64')
Int64Index([220], dtype='int64')
Int64Index([832], dtype='int64')
Int64Index([1519], dtype='int64')
Int64Index([1595], dtype='int64')
Int64Index([1860], dtype='int64')
Int64Index([602], dtype='int64')
Int64Index([1204], dtype='int64')
Int64Index([279], dtype='int64')
Int64Index([1682], dtype='int64')
Int64Index([33], dtype='int64')
Int64Index([1626], dtype='int64')
Int64Index([112], dtype='int64')
Int64Index([1790], dtype='int64')
Int64Index([1505], dtype='int64')
Int64Index([2016], dtype='int64')
Int64Index([783], dtype='int64')
Int64Index([1787], dtype='int64')
Int64Index([558], dtype='int64')
Int64Index([197], dtype='int64')
Int64Index([1320], dtype='int64')
Int64Index([1324], dtype='int64')
Int64Index([2083], dtype='int64')
Int64Index([1300], dtype='int64')
Int64Index([1966], dtype='int64')
Int64Index([1422], dtype='int64')
Int64Index([508], dtype='int64')
Int64Index([1633], dtype='int64')
Int64Index([1941], dtype='int64')
Int64Index([968], dtype='int64')
Int64Index([2065], dtype='int64')
Int64Index([2055], dtype='int64')
Int64Index([1779], dtype='int64')
Int64Index([1613], dtype='int64')
Int64Index([2037], dtype='int64')
Int64Index([388], dtype='int64')
Int64Index([579], dtype='int64')
Int64Index([30], dtype='int64')
Int64Index([1013], dtype='int64')
Int64Index([848], dtype='int64')
Int64Index([1278], dtype='int64')
Int64Index([1464], dtype='int64')
Int64Index([1728], dtype='int64')
Int64Index([565], dtype='int64')
Int64Index([1885], dtype='int64')
Int64Index([838], dtype='int64')
Int64Index([1858], dtype='int64')
Int64Index([684], dtype='int64')
Int64Index([1630], dtype='int64')
Int64Index([141], dtype='int64')
Int64Index([836], dtype='int64')
Int64Index([349], dtype='int64')
Int64Index([8], dtype='int64')
Int64Index([1625], dtype='int64')
Int64Index([1631], dtype='int64')
Int64Index([1992], dtype='int64')
Int64Index([952], dtype='int64')
Int64Index([182], dtype='int64')
Int64Index([535], dtype='int64')
Int64Index([1395], dtype='int64')
Int64Index([1602], dtype='int64')
Int64Index([475], dtype='int64')
Int64Index([423], dtype='int64')
Int64Index([1237], dtype='int64')
Int64Index([962], dtype='int64')
Int64Index([1349], dtype='int64')
Int64Index([623], dtype='int64')
Int64Index([726], dtype='int64')
Int64Index([904], dtype='int64')
Int64Index([59], dtype='int64')
Int64Index([2074], dtype='int64')
Int64Index([50], dtype='int64')
Int64Index([1255], dtype='int64')
Int64Index([800], dtype='int64')
Int64Index([1433], dtype='int64')
Int64Index([1727], dtype='int64')
Int64Index([386], dtype='int64')
Int64Index([1678], dtype='int64')
Int64Index([198], dtype='int64')
Int64Index([118], dtype='int64')
Int64Index([1976], dtype='int64')
Int64Index([1450], dtype='int64')
Int64Index([1081], dtype='int64')
Int64Index([1930], dtype='int64')
Int64Index([2004], dtype='int64')
Int64Index([738], dtype='int64')
Int64Index([68], dtype='int64')
Int64Index([835], dtype='int64')
Int64Index([194], dtype='int64')
Int64Index([2090], dtype='int64')
Int64Index([12], dtype='int64')
Int64Index([1592], dtype='int64')
Int64Index([117], dtype='int64')
Int64Index([1463], dtype='int64')
Int64Index([778], dtype='int64')
Int64Index([1947], dtype='int64')
Int64Index([387], dtype='int64')
Int64Index([1828], dtype='int64')
Int64Index([809], dtype='int64')
Int64Index([1348], dtype='int64')
Int64Index([1240], dtype='int64')
Int64Index([448], dtype='int64')
Int64Index([1074], dtype='int64')
Int64Index([1916], dtype='int64')
Int64Index([1163], dtype='int64')
Int64Index([1662], dtype='int64')
Int64Index([1496], dtype='int64')
Int64Index([107], dtype='int64')
Int64Index([1629], dtype='int64')
Int64Index([181], dtype='int64')
Int64Index([1213], dtype='int64')
Int64Index([1535], dtype='int64')
Int64Index([1390], dtype='int64')
Int64Index([1874], dtype='int64')
Int64Index([2026], dtype='int64')
Int64Index([765], dtype='int64')
Int64Index([1932], dtype='int64')
Int64Index([1344], dtype='int64')
Int64Index([638], dtype='int64')
Int64Index([1618], dtype='int64')
Int64Index([1226], dtype='int64')
Int64Index([1632], dtype='int64')
Int64Index([75], dtype='int64')
Int64Index([188], dtype='int64')
Int64Index([1232], dtype='int64')
Int64Index([1446], dtype='int64')
Int64Index([1615], dtype='int64')
Int64Index([137], dtype='int64')
Int64Index([1480], dtype='int64')
Int64Index([932], dtype='int64')
Int64Index([1530], dtype='int64')
Int64Index([524], dtype='int64')
Int64Index([612], dtype='int64')
Int64Index([247], dtype='int64')
Int64Index([1346], dtype='int64')
Int64Index([1962], dtype='int64')
Int64Index([1777], dtype='int64')
Int64Index([1771], dtype='int64')
Int64Index([1834], dtype='int64')
Int64Index([318], dtype='int64')
Int64Index([752], dtype='int64')
Int64Index([1071], dtype='int64')
Int64Index([1695], dtype='int64')
Int64Index([1380], dtype='int64')
Int64Index([1830], dtype='int64')
Int64Index([15], dtype='int64')
Int64Index([2023], dtype='int64')
Int64Index([1534], dtype='int64')
Int64Index([1901], dtype='int64')
Int64Index([2097], dtype='int64')
Int64Index([331], dtype='int64')
Int64Index([799], dtype='int64')
Int64Index([834], dtype='int64')
Int64Index([300], dtype='int64')
Int64Index([1416], dtype='int64')
Int64Index([1897], dtype='int64')
Int64Index([1692], dtype='int64')
Int64Index([1636], dtype='int64')
Int64Index([805], dtype='int64')
Int64Index([1091], dtype='int64')
Int64Index([951], dtype='int64')
Int64Index([1532], dtype='int64')
Int64Index([1896], dtype='int64')
Int64Index([811], dtype='int64')
Int64Index([1506], dtype='int64')
Int64Index([913], dtype='int64')
Int64Index([618], dtype='int64')
Int64Index([2007], dtype='int64')
Int64Index([119], dtype='int64')
Int64Index([27], dtype='int64')
Int64Index([485], dtype='int64')
Int64Index([1361], dtype='int64')
Int64Index([1129], dtype='int64')
Int64Index([1873], dtype='int64')
Int64Index([1001], dtype='int64')
Int64Index([1377], dtype='int64')
Int64Index([1156], dtype='int64')
Int64Index([191], dtype='int64')
Int64Index([1120], dtype='int64')
Int64Index([2031], dtype='int64')
Int64Index([1363], dtype='int64')
Int64Index([254], dtype='int64')
Int64Index([1898], dtype='int64')
Int64Index([688], dtype='int64')
Int64Index([61], dtype='int64')
Int64Index([1136], dtype='int64')
Int64Index([64], dtype='int64')
Int64Index([1514], dtype='int64')
Int64Index([526], dtype='int64')
Int64Index([53], dtype='int64')
Int64Index([1591], dtype='int64')
Int64Index([1266], dtype='int64')
Int64Index([1810], dtype='int64')
Int64Index([703], dtype='int64')
Int64Index([1522], dtype='int64')
Int64Index([916], dtype='int64')
Int64Index([1202], dtype='int64')
Int64Index([1340], dtype='int64')
Int64Index([54], dtype='int64')
Int64Index([1718], dtype='int64')
Int64Index([620], dtype='int64')
Int64Index([1808], dtype='int64')
Int64Index([925], dtype='int64')
Int64Index([87], dtype='int64')
Int64Index([586], dtype='int64')
Int64Index([596], dtype='int64')
Int64Index([1046], dtype='int64')
Int64Index([462], dtype='int64')
Int64Index([1359], dtype='int64')
Int64Index([1683], dtype='int64')
Int64Index([421], dtype='int64')
Int64Index([395], dtype='int64')
Int64Index([817], dtype='int64')
Int64Index([1785], dtype='int64')
Int64Index([426], dtype='int64')
Int64Index([1862], dtype='int64')
Int64Index([529], dtype='int64')
Int64Index([1982], dtype='int64')
Int64Index([1215], dtype='int64')
Int64Index([1793], dtype='int64')
Int64Index([731], dtype='int64')
Int64Index([1113], dtype='int64')
Int64Index([391], dtype='int64')
Int64Index([1174], dtype='int64')
Int64Index([1919], dtype='int64')
Int64Index([1998], dtype='int64')
Int64Index([1895], dtype='int64')
Int64Index([2088], dtype='int64')
Int64Index([816], dtype='int64')
Int64Index([39], dtype='int64')
Int64Index([1250], dtype='int64')
Int64Index([253], dtype='int64')
Int64Index([495], dtype='int64')
Int64Index([1225], dtype='int64')
Int64Index([1431], dtype='int64')
Int64Index([717], dtype='int64')
Int64Index([1931], dtype='int64')
Int64Index([1187], dtype='int64')
Int64Index([1811], dtype='int64')
Int64Index([1475], dtype='int64')
Int64Index([675], dtype='int64')
Int64Index([1843], dtype='int64')
Int64Index([1271], dtype='int64')
Int64Index([1669], dtype='int64')
Int64Index([1994], dtype='int64')
Int64Index([1040], dtype='int64')
Int64Index([748], dtype='int64')
Int64Index([802], dtype='int64')
Int64Index([80], dtype='int64')
Int64Index([1021], dtype='int64')
Int64Index([1799], dtype='int64')
Int64Index([1974], dtype='int64')
Int64Index([169], dtype='int64')
Int64Index([995], dtype='int64')
Int64Index([2025], dtype='int64')
Int64Index([701], dtype='int64')
Int64Index([1298], dtype='int64')
Int64Index([691], dtype='int64')
Int64Index([2003], dtype='int64')
Int64Index([1960], dtype='int64')
Int64Index([1704], dtype='int64')
Int64Index([81], dtype='int64')
Int64Index([660], dtype='int64')
Int64Index([1802], dtype='int64')
Int64Index([1967], dtype='int64')
Int64Index([431], dtype='int64')
Int64Index([121], dtype='int64')
Int64Index([967], dtype='int64')
Int64Index([491], dtype='int64')
Int64Index([347], dtype='int64')
Int64Index([484], dtype='int64')
Int64Index([1838], dtype='int64')
Int64Index([1730], dtype='int64')
Int64Index([865], dtype='int64')
Int64Index([248], dtype='int64')
Int64Index([1548], dtype='int64')
Int64Index([959], dtype='int64')
Int64Index([404], dtype='int64')
Int64Index([1504], dtype='int64')
Int64Index([1041], dtype='int64')
Int64Index([1307], dtype='int64')
Int64Index([1903], dtype='int64')
Int64Index([246], dtype='int64')
Int64Index([134], dtype='int64')
Int64Index([763], dtype='int64')
Int64Index([1503], dtype='int64')
Int64Index([160], dtype='int64')
Int64Index([1980], dtype='int64')
Int64Index([1850], dtype='int64')
Int64Index([78], dtype='int64')
Int64Index([1783], dtype='int64')
Int64Index([653], dtype='int64')
Int64Index([398], dtype='int64')
Int64Index([592], dtype='int64')
Int64Index([761], dtype='int64')
Int64Index([1693], dtype='int64')
Int64Index([562], dtype='int64')
Int64Index([1648], dtype='int64')
Int64Index([342], dtype='int64')
Int64Index([938], dtype='int64')
Int64Index([1756], dtype='int64')
Int64Index([572], dtype='int64')
Int64Index([784], dtype='int64')
Int64Index([1599], dtype='int64')
"""
import re

lines = input_string.strip().split('\n')
indexes_to_drop = [int(line.split('[')[1].split(']')[0]) for line in lines]
print(indexes_to_drop)

for index in indexes_to_drop:
    if index in muse_data_step.index:
        muse_data_step = muse_data_step.drop(index)

[539, 1652, 1796, 2038, 570, 1313, 1088, 927, 1690, 1510, 1773, 710, 1145, 422, 1083, 303, 928, 1169, 362, 382, 814, 1183, 625, 2029, 1703, 854, 1357, 1123, 544, 747, 807, 205, 657, 891, 690, 1760, 171, 1780, 1292, 488, 277, 1280, 1934, 1781, 795, 521, 1722, 1764, 338, 1110, 482, 564, 813, 1019, 1465, 1825, 1642, 1950, 857, 291, 1929, 1681, 1036, 1622, 74, 1241, 1772, 806, 1924, 1263, 1247, 705, 200, 436, 339, 1749, 1003, 1256, 1979, 1697, 1223, 328, 260, 1396, 587, 1915, 979, 354, 1521, 1517, 1273, 332, 439, 288, 383, 1430, 941, 1098, 1926, 1972, 442, 1418, 446, 1235, 1593, 289, 1956, 1502, 825, 24, 1646, 2080, 1009, 819, 766, 1230, 1102, 336, 568, 35, 1326, 950, 1130, 1536, 5, 410, 1253, 359, 676, 982, 1813, 622, 1165, 1596, 892, 249, 1921, 1826, 40, 1392, 1958, 244, 1117, 10, 1893, 460, 534, 2018, 1761, 1748, 792, 538, 885, 719, 267, 1908, 1281, 1719, 363, 637, 1643, 1701, 1408, 1135, 843, 1425, 610, 1243, 609, 677, 574, 1497, 1080, 507, 489, 1197, 1092, 1269, 696, 1731, 1507, 313, 

In [33]:
display(muse_data_step)

,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
2,1,1,4,126359.50607515374,18,1.0,1,1,1.0,19.0,2,4.0
3,1,1,4,75127.85895569899,47,8.0,4,1,1.0,16.0,2,4.0
6,4,2,3,35818.3718385832,12,None,1,1,None,9.0,2,4.0
7,1,1,3,67608.02336629394,36,1.0,1,1,None,14.0,2,2.0
13,1,2,4,-8500.0,45,1.0,1,1,4.0,18.0,2,4.0
14,1,1,4,None,55,1.0,1,1,6.0,14.0,2,None
18,4,2,3,-8500.0,36,1.0,1,1,1.0,21.0,2,4.0
23,9,1,4,5687.475002899468,31,6.0,1,1,6.0,16.0,2,4.0
25,9,2,4,None,8,1.0,1,1,1.0,21.0,2,4.0
29,1,1,4,-8500.0,47,None,1,1,6.0,11.0,2,None


In [34]:
muse_data_step.to_csv('datasets/muse/updated_muse_data_step.csv',index=False)
muse_data_step[muse_data_step['RAC1P']==7]

,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL


In [35]:
from platform import python_version

print(python_version())

3.6.13
